In [2]:
from sklearn.cross_validation import KFold
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
import numpy as np
import pylab as pl
import pandas as pd

C:\Users\erica\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


### 2. a.Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity.

In [1]:
def distance(x, data):
    norm_data = np.linalg.norm(data)
    norm_x = np.linalg.norm(x)
    similarity = np.dot(data, x)/(norm_data * norm_x)
    distance = 1 - similarity
    return distance

### b.Load the data set  Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [3]:
np.set_printoptions(suppress=True, precision=2, linewidth=120)

In [4]:
import os
os.chdir('C:\\Users\\erica\\OneDrive for Business\\01 Current Classes\\DSC478 - Machine Programming\\Assignment3')

In [9]:
news_data = pd.DataFrame(np.genfromtxt ("matrix.txt", delimiter=",", dtype = int).T)

In [12]:
news_data.shape

(2500, 9328)

In [38]:
terms = (np.genfromtxt ("terms.txt", delimiter="\t", dtype = str))
terms.shape

(9328L,)

In [93]:
classes = pd.DataFrame(np.genfromtxt("classes.txt", delimiter=' ',dtype=int,skip_header=1))
classes
#drop 0 column
classes = classes.drop(classes.columns[0], axis = 1)
classes.head()
classes = np.array(classes)
classes.shape

(2500L, 1L)

In [25]:
#Split data into train and test
from sklearn.cross_validation import train_test_split
x_train, x_test, classes_train, classes_test = train_test_split(news_data, classes, test_size = 0.2, random_state = 33)

In [27]:
x_train.shape

(2000, 9328)

In [36]:
DF = np.array([(news_data!=0).sum(0)])
print DF

[[10  6 22 ...  3  3  4]]


In [37]:
NDocs = len(news_data[0])
print NDocs

2500


In [40]:
nmatrix = np.ones(np.shape(news_data), dtype=float)*NDocs

In [48]:
IDF = np.log2(np.divide(nmatrix, DF))
print IDF[0:2,]

[[7.97 8.7  6.83 ... 9.7  9.7  9.29]
 [7.97 8.7  6.83 ... 9.7  9.7  9.29]]


In [51]:
DT_tfidf = news_data * IDF


In [53]:
from sklearn.cross_validation import train_test_split
x_train, x_test, classes_train, classes_test = train_test_split(DT_tfidf, classes, test_size = 0.2, random_state = 33)

In [59]:
## perfornm KMeans
from sklearn.cluster import KMeans 


### c.Perform Kmeans clustering on the training data.

In [62]:
## perfornm KMeans
from sklearn.cluster import KMeans 
from numpy import *

def distEuclid(vecA, vecB):
    return sqrt(sum(power(vecA - vecB, 2))) #la.norm(vecA-vecB)

def randCent(dataSet, k):
    n = shape(dataSet)[1]
    centroids = zeros((k,n), dtype=float)
    for j in range(n): #create random cluster centers
        minJ = min(dataSet[:,j])
        rangeJ = float(max(dataSet[:,j]) - minJ)
        centroids[:,j] = minJ + rangeJ * random.rand(k)
    return centroids 

def kMeans(dataSet, k, distMeas=distance, createCent=randCent):
    m = shape(dataSet)[0]
    clusterAssment = zeros((m,2))#create mat to assign data points 
                                      #to a centroid, also holds SE of each point
    centroids = createCent(dataSet, k)
    clusterChanged = True
    while clusterChanged:
        clusterChanged = False
        for i in range(m):#for each data point assign it to the closest centroid
            minDist = inf; minIndex = -1
            for j in range(k):
                distJI = distMeas(centroids[j,:],dataSet[i,:])
                if distJI < minDist:
                    minDist = distJI; minIndex = j
            if clusterAssment[i,0] != minIndex: clusterChanged = True
            clusterAssment[i,:] = minIndex,minDist**2
        # print centroids
        for cent in range(k):#recalculate centroids
            ptsInClust = dataSet[nonzero(clusterAssment[:,0]==cent)[0]] #get all the point in this cluster - Note: this was incorrect in the original distribution.
            if(len(ptsInClust)!=0):
                centroids[cent,:] = mean(ptsInClust, axis=0) #assign centroid to mean - Note condition was added 10/28/2013
    return centroids, clusterAssment


In [68]:
DT_tfidf_train = np.array(x_train)
centroids_tfidf, clusters_tfidf = kMeans(DT_tfidf_train, 3)


In [66]:
print "\t\tCluster0\tCluster1\tCluster2"
for i in range(len(terms)):
    print "%10s\t%.4f\t\t%.4f\t\t%.4f" %(terms[i],centroids_tfidf[0][i],centroids_tfidf[1][i],centroids_tfidf[2][i])

		Cluster0	Cluster1	Cluster2
        aa	0.0392		0.0594		0.0000
     aargh	0.0000		0.0354		0.0000
     aaron	0.0470		0.3472		0.0000
    aaronc	0.0000		0.0660		0.0000
        ab	0.0523		0.1132		0.8430
   abandon	0.0160		0.0440		0.0000
       abc	0.0126		0.2336		0.0000
      abid	0.0000		0.0263		0.0000
      abil	0.0809		0.2341		0.0000
       abl	0.2622		0.3744		0.0000
    abomin	0.0000		0.0441		0.0000
     abort	0.0000		0.0197		0.0000
    abound	0.0000		0.0304		0.0000
   abraham	0.0000		0.0460		0.0000
    abridg	0.0000		0.0252		0.0000
    abroad	0.0000		0.0378		0.0000
    absenc	0.0000		0.0660		0.0000
    absent	0.0000		0.0402		0.0000
   absolut	0.0859		0.3698		0.0000
   abstain	0.0000		0.0413		0.0000
    abstin	0.0000		0.0756		0.0000
  abstract	0.0303		0.0522		0.0000
    absurd	0.0167		0.0460		0.0000
     abund	0.0000		0.0252		0.0000
      abus	0.0000		0.2441		0.0000
        ac	0.1479		0.0576		0.0000
    academ	0.0078		0.2646		0.0000
   academi	0.0183		0.0189		0.0000
  academia	0.0000		

     crane	0.0618		0.0000		0.0000
      crap	0.0283		0.0732		0.0000
     crash	0.4862		0.0386		0.0000
    craven	0.0163		0.0506		0.0000
  crawford	0.0191		0.0197		0.0000
     crawl	0.0183		0.0252		0.0000
   craycom	0.0000		0.0690		0.0000
     crazi	0.0490		0.0627		0.0000
       crc	0.0095		0.0263		0.0000
     creas	0.0000		0.0649		0.0000
     creat	0.2314		0.2614		0.0000
  creation	0.0251		0.1601		0.0000
   creativ	0.0000		0.0252		0.0000
   creator	0.0227		0.0522		0.0000
   creatur	0.0086		0.0295		0.0000
   credibl	0.0000		0.0731		0.0000
    credit	0.0523		0.1126		0.0000
     creed	0.0000		0.0899		0.0000
 creighton	0.0000		0.0263		0.0000
      crep	0.0000		0.1292		0.0000
     crest	0.0000		0.0197		0.0000
      crew	0.0000		0.0304		0.0000
       cri	0.0000		0.0547		0.0000
     crime	0.0058		0.2504		0.0000
    crimin	0.0000		0.2211		0.0000
     cring	0.0265		0.0182		0.0000
    crippl	0.0295		0.0710		0.0000
     crisi	0.0077		0.0584		0.0000
     crisp	0.0191		0.0066		0.0000
  criteria	0.0

   freeman	0.0477		0.0000		0.0000
freenetcarletonca	0.0080		0.0826		0.0000
   freewar	0.0835		0.0057		0.0000
    french	0.0078		0.0486		0.0000
  frequenc	0.0627		0.0216		0.0000
  frequent	0.0257		0.1417		0.0000
     fresh	0.0154		0.0531		0.0000
       fri	0.0428		0.0059		0.0000
    fridai	0.0221		0.0710		0.0000
  friedman	0.0368		0.0609		0.0000
    friend	0.2236		0.3169		0.0000
  friendli	0.0163		0.0337		0.0000
friendship	0.0000		0.0862		0.0000
   friesen	0.0000		0.0304		0.0000
  frighten	0.0000		0.0402		0.0000
frighteningli	0.0095		0.0197		0.0000
     fring	0.0000		0.0252		0.0000
     fritz	0.0382		0.0066		0.0000
       fro	0.0091		0.0189		0.0000
      frog	0.0706		0.0365		0.0000
     front	0.1882		0.1728		0.0000
   frontal	0.0086		0.0295		0.0000
  frontier	0.0176		0.1033		0.0000
    frozen	0.0083		0.0345		0.0000
     fruit	0.0163		0.0337		0.0000
  fruition	0.0000		0.0197		0.0000
  frustrat	0.0299		0.0514		0.0000
        fs	0.0088		0.0729		0.0000
fslnoaagov	0.0000		0.0304		0.0000
    

maxcarletonca	0.0382		0.0000		0.0000
     maxim	0.0095		0.0132		0.0000
   maximum	0.0559		0.0165		0.0000
    maxtor	0.1169		0.0000		0.0000
    maxwel	0.0088		0.0243		0.0000
      maxx	0.0286		0.0000		0.0000
      mayn	0.0000		0.0526		0.0000
   maynard	0.0000		0.3603		0.0000
      mayo	0.0091		0.0378		0.0000
     mazda	0.0668		0.0000		0.0000
        mb	0.2352		0.2686		9.0823
     mbeck	0.0366		0.0000		0.0000
  mbeckman	0.0000		0.0329		0.0000
mbeckmanmbeckmancom	0.0000		0.0263		0.0000
      mbhj	0.0000		0.0000		4.3123
       mbj	0.0000		0.0000		6.4685
       mbn	0.0000		0.1053		0.0000
      mbxn	0.0000		0.0059		19.3394
     mbxom	0.0000		0.0000		4.3123
        mc	0.0299		0.3035		10.1164
       mca	0.0191		0.0066		0.0000
       mcc	0.0000		0.0402		0.0000
     mccoi	0.0091		0.0441		0.0000
  mcdonald	0.0000		0.0562		0.0000
   mcdowel	0.0000		0.0252		0.0000
 mceachern	0.0000		0.0486		0.0000
  mcfadyen	0.0382		0.0000		0.0000
    mcgill	0.0000		0.0805		0.0000
 mcgrawhil	0.0191		0.0132		0.0000


       rod	0.0000		0.1080		0.0000
    rodnei	0.0191		0.0066		0.0000
   roenick	0.0080		0.0771		0.0000
     roger	0.1055		0.3995		0.0000
      rogu	0.0000		0.0547		0.0000
       roi	0.0000		0.2456		0.0000
    roland	0.0399		0.0550		0.0000
      role	0.0131		0.1531		0.0000
      rolf	0.0000		0.1070		0.0000
      roll	0.1076		0.0395		0.0000
    roller	0.0382		0.0066		0.0000
       rom	0.0900		0.1152		0.0000
     roman	0.0152		0.3692		0.0000
  romanbmp	0.0077		0.0212		6.0887
      rome	0.0000		0.1235		0.0000
       ron	0.1055		0.2942		0.0000
     ronni	0.0000		0.0252		0.0000
      roof	0.0343		0.0118		0.0000
     rooki	0.0373		0.0772		0.0000
      room	0.0732		0.1177		0.0000
      root	0.0336		0.0972		0.0000
      rope	0.0191		0.0132		0.0000
      rose	0.0726		0.1045		0.0000
   rosecom	0.0000		0.0315		0.0000
  rosemail	0.0000		0.0197		0.0000
   rosenet	0.0000		0.0197		0.0000
  roseread	0.0000		0.0197		0.0000
   rospach	0.0000		0.0441		0.0000
      ross	0.0693		0.0668		0.0000
    roster	0.0

      wire	0.0436		0.0600		0.0000
   wiretap	0.0000		0.5058		0.0000
wiretaplik	0.0000		0.0197		0.0000
wisconsinmadison	0.0095		0.0132		0.0000
    wisdom	0.0063		0.1514		0.0000
      wise	0.0771		0.0820		0.0000
     wiseb	0.0573		0.0000		0.0000
      wish	0.1071		0.2247		0.0000
       wit	0.0117		0.2063		1.9886
     witch	0.0286		0.0197		0.0000
  withhold	0.0086		0.0354		0.0000
wittgenstein	0.0000		0.0460		0.0000
      wive	0.0000		0.0590		0.0000
    wizard	0.0685		0.0000		0.0000
        wj	0.0091		0.0504		0.0000
       wjr	0.0000		0.0460		0.0000
        wk	0.0088		0.0790		0.0000
        wl	0.0000		0.0315		2.0639
  wlieftin	0.0286		0.0000		0.0000
        wm	0.0073		0.2302		105.7918
     wmbxn	0.0000		0.0000		13.4156
       wmv	0.0000		0.0263		0.0000
        wn	0.0088		0.0912		0.0000
   wnkretz	0.0731		0.0000		0.0000
        wo	0.0000		0.0819		3.0959
      wolv	0.0088		0.0365		0.0000
  wolverin	0.3971		0.0000		0.0000
     woman	0.0361		0.1825		0.0000
     women	0.0195		0.1742		0.0000
   

In [84]:
def output(data, K, numterms):
    centroids_tfidf, clusters_tfidf = kMeans(data, K)
    for i in range(K):
        print"Cluster", i+1, ":"
        cluster_term = data[clusters_tfidf[:,0]==i]
        print"Number of documents in Cluster", cluster_term.shape[0]
        cluster_doc_freq = np.array([(cluster_term.T!=0).sum(1)]).T
        term_doc_cluster = cluster_doc_freq/float(cluster_term.shape[0])
        term_doc_cluster = map(list, term_doc_cluster)
        term_doc_cluster = [elem[0] for elem in term_doc_cluster]
        cluster_doc_freq = [elem[0] for elem in cluster_doc_freq]
        a=sorted(zip(terms, cluster_doc_freq, term_doc_cluster),key=lambda x:x[2],reverse=True) #Sort in decreasing order
        print 'Word: ' , '\tFrequency: ', '\t% '
        for elem in a[:numterms]:
            ##print 'Word: ' , '\tFrequency: ', '\t% '
            print elem[0], "\t" ,elem[1],  "\t\t", elem[2]*100
        print'\n'
    return centroids_tfidf, clusters_tfidf

In [85]:
output(DT_tfidf_train, 5, 10)

Cluster 1 :
Number of documents in Cluster 392
Word:  	Frequency:  	% 
write 	244 		62.244897959183675
on 	219 		55.86734693877551
god 	214 		54.59183673469388
articl 	187 		47.704081632653065
christian 	179 		45.66326530612245
peopl 	174 		44.38775510204081
think 	169 		43.11224489795919
know 	166 		42.3469387755102
time 	150 		38.265306122448976
see 	148 		37.755102040816325


Cluster 2 :
Number of documents in Cluster 406
Word:  	Frequency:  	% 
write 	221 		54.43349753694581
game 	217 		53.44827586206896
team 	193 		47.53694581280788
go 	177 		43.5960591133005
articl 	172 		42.364532019704434
plai 	165 		40.64039408866995
hockei 	159 		39.16256157635468
on 	148 		36.45320197044335
get 	146 		35.960591133004925
think 	125 		30.78817733990148


Cluster 3 :
Number of documents in Cluster 404
Word:  	Frequency:  	% 
write 	273 		67.57425742574257
articl 	207 		51.23762376237624
clipper 	185 		45.79207920792079
encrypt 	185 		45.79207920792079
kei 	185 		45.79207920792079
on 	181 		44.8

(array([[0.06, 0.  , 1.13, ..., 0.05, 0.02, 0.14],
        [0.  , 0.09, 0.02, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 0.  , 0.  , 0.  ],
        [0.04, 0.  , 0.06, ..., 0.  , 0.  , 0.  ],
        [0.  , 0.  , 0.  , ..., 1.21, 0.  , 0.  ]]), array([[3.  , 0.88],
        [1.  , 0.9 ],
        [3.  , 0.81],
        ...,
        [0.  , 0.4 ],
        [3.  , 0.63],
        [2.  , 0.65]]))

### d.Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [86]:
print centroids_tfidf

[[1.77 0.   0.   ... 2.16 1.08 6.19]
 [0.   0.   0.   ... 1.21 0.   0.  ]
 [0.02 0.02 0.25 ... 0.   0.   0.  ]]


In [106]:
clusters = clusters_tfidf.T[0]
clusters = clusters.astype(int)
print clusters


[2 2 2 ... 2 2 2]


In [99]:
classes_train2 = np.array(classes_train)
classes_train2.shape

(2000L, 1L)

In [109]:
from sklearn.metrics import completeness_score, homogeneity_score
#print completeness_score(classes_train,clusters)
completeness = completeness_score(np.array(classes_train2.T)[0],clusters)
homogeneity = homogeneity_score(np.array(classes_train.T)[0],clusters)

print "Completeness: ", completeness
print "Homogeneity: ", homogeneity

Completeness:  0.11285631067090174
Homogeneity:  0.0038477562766994914


### e.Finally, using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and cluster centroids. For each test document show the predicted class label as well as Cosine similarity to the corresponding cluster